In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
!pip install transformers

In [4]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [31]:
tokens = tokenizer.encode('It was mazing,I loved it. Great', return_tensors='pt')
# convert the string into tokens, return_tensors is the keyword argument in which we have used pt stands for pytorch

In [32]:
tokens

tensor([[  101, 10197, 10140, 66813, 10285,   117,   151, 46747, 10197,   119,
         11838,   102]])

In [34]:
tokenizer.decode(tokens[0])

'[CLS] it was mazing, i loved it. great [SEP]'

In [35]:
result = model(tokens)

In [36]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.5173, -2.3455, -0.7136,  1.6171,  3.0845]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [37]:
result.logits

tensor([[-2.5173, -2.3455, -0.7136,  1.6171,  3.0845]],
       grad_fn=<AddmmBackward0>)

In [38]:
int(torch.argmax(result.logits))+1

5

In [41]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment*.')
results = soup.find_all('p',{'class' :regex})
reviews = [result.text for result in results]

In [42]:
reviews[0]

'The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

In [44]:
#data frame
import numpy as np
import pandas as pd

df = pd.DataFrame(np.array(reviews), columns=['reviews'])

In [45]:
df.head()

,reviews
0,The food is fresh and tasty. The scallop cevi...
1,The food was decent not great.. We had the gu...
2,"Food was okay, guacamole was below average. Se..."
3,The food and service here was really good. It...
4,Hotter than I imagine myself to be and firing ...


In [47]:
df['reviews'].iloc[0]

'The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

In [52]:
def sentiment_score(review):
  tokens = tokenizer.encode(review,return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [56]:
sentiment_score(df['reviews'].iloc[0])

4

In [57]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [60]:
df

,reviews,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Hotter than I imagine myself to be and firing ...,5
5,Visiting from Texas and decided to give this r...,5
6,Don't come here expecting legit Mexican food b...,3
7,Out of all the restaurants that I tried in Syd...,5
8,"Great atmosphere, attentive service, solid mar...",3
9,We came here on a Thursday night @ 5pm and by ...,4


In [62]:
df['reviews'].iloc[3]

'The food and service here was really good.  It was more like tapas food than Mexican food!  The drinks were amazing too!'